In [2]:
%pip install esgf-pyclient==0.3.1

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### listagem de variáveis do CMPI6

In [1]:
import sys

# apontar para a versão local da lib
sys.path.insert(0, './dreqPy')

In [2]:
from dreqPy import dreq

In [3]:
# inicializando a biblioteca
dq = dreq.loadDreq()

In [4]:
# listando variáveis relacionadas à temperatura
temperature_vars = [var for var in dq.coll["var"].items if "temperature" in var.title.lower()]

for v in temperature_vars:
    print(v.label, "-", v.title, "-", v.sn)

bigthetao - Sea Water Conservative Temperature - sea_water_conservative_temperature
bigthetaoga - Global Average Sea Water Conservative Temperature - sea_water_conservative_temperature
edt - Eddy Diffusivity Coefficient for Temperature - atmosphere_heat_diffusivity
hfevapds - Temperature Flux Due to Evaporation Expressed as Heat Flux out of Sea Water - temperature_flux_due_to_evaporation_expressed_as_heat_flux_out_of_sea_water
hfrainds - Temperature Flux Due to Rainfall Expressed as Heat Flux into Sea Water - temperature_flux_due_to_rainfall_expressed_as_heat_flux_into_sea_water
hfrunoffds - Temperature Flux Due to Runoff Expressed as Heat Flux into Sea Water - temperature_flux_due_to_runoff_expressed_as_heat_flux_into_sea_water
litempbotfl - Basal Temperature of Floating Ice Shelf - land_ice_basal_temperature
litempbotgr - Basal Temperature of Grounded Ice Sheet - land_ice_basal_temperature
litemptop - Temperature at Top of Ice Sheet Model - temperature_at_top_of_ice_sheet_model
litem

### acesso ao ESGF

In [8]:
from collections import Counter, defaultdict
import logging
from pyesgf.search import SearchConnection

In [6]:
logging.getLogger('pyesgf.search').setLevel(logging.ERROR)

In [7]:
# variáveis relevantes de acordo com GCOS
variaveis = ["tas", "tasmax", "tasmin", "ps", "pr", "hur", "rsds", "rlut", "uas", "vas", "tos", "sos", "sic"]

In [55]:
conn = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False)   # distrib = False —> força a busca apenas no LLNL, que costuma estar mais estável

ctx = conn.new_context(
    project="CMIP6",
    experiment_id=["historical", "ssp245", "ssp585"],
    variable_id=variaveis,
    frequency=["mon"],
    #member_id="r1i1p1f1",
    #grid_label="gn",
    replica=False,
    latest=True
)

# armazenando os resultados
results = list(ctx.search(ignore_facet_check=True))

# extraindo os modelos (source_id)
models = [d.json.get("source_id", [None])[0] for d in results if "source_id" in d.json]

# frequência de cada modelo
print(Counter(models))

"""
mapping = {}

for d in ctx.search(ignore_facet_check=True):
    models = [d.json.get("source_id", [None])[0] for d in ctx.search(ignore_facet_check=True)]
    print(Counter(models))

    model = d.json.get("source_id", [None])[0]
    experiment = d.json.get("experiment_id", [None])[0]
    variable = d.json.get("variable_id", [None])[0]
    
    if model and experiment and variable:
        mapping.setdefault(model, {}).setdefault(experiment, set()).add(variable)

# Exibir resultados
for model, exps in mapping.items():
    print(f"\nModelo: {model}")
    for exp, vars in exps.items():
        print(f"  Experimento: {exp} → Variáveis: {sorted(vars)}")"""

Counter()


'\nmapping = {}\n\nfor d in ctx.search(ignore_facet_check=True):\n    models = [d.json.get("source_id", [None])[0] for d in ctx.search(ignore_facet_check=True)]\n    print(Counter(models))\n\n    model = d.json.get("source_id", [None])[0]\n    experiment = d.json.get("experiment_id", [None])[0]\n    variable = d.json.get("variable_id", [None])[0]\n\n    if model and experiment and variable:\n        mapping.setdefault(model, {}).setdefault(experiment, set()).add(variable)\n\n# Exibir resultados\nfor model, exps in mapping.items():\n    print(f"\nModelo: {model}")\n    for exp, vars in exps.items():\n        print(f"  Experimento: {exp} → Variáveis: {sorted(vars)}")'

In [57]:
variaveis = ["tas", "tasmin", "tasmax", "tos", "ps", "pr"]  # exemplo com 6 variáveis
modelos_por_variavel = defaultdict(set)

conn = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False)

for var in variaveis:
    ctx = conn.new_context(
        project="CMIP6",
        experiment_id=["historical", "ssp245", "ssp585"],
        variable_id=var,
        frequency="mon",
        replica=False,
        latest=True
    )

    results = list(ctx.search(ignore_facet_check=True))
    for d in results:
        modelo = d.json.get("source_id", [None])[0]
        if modelo:
            modelos_por_variavel[var].add(modelo)

# Exibindo os resultados
for var, modelos in modelos_por_variavel.items():
    print(f"{var}: {sorted(modelos)}")

KeyboardInterrupt: 

In [ ]:
EC-Earth3 (1401)
CanESM5 (1300)
MIROC6 (1201)
ACCESS-ESM1-5 (760)
MPI-ESM1-2-LR (709)
GISS-E2-1-G (561)
MIROC-ES2L (558)
CanESM5-1 (512)
IPSL-CM6A-LR (397)
CNRM-CM6-1 (297)
UKESM1-0-LL (286)
MRI-ESM2-0 (216)
GISS-E2-1-H (199)
EC-Earth3-Veg (193)
ACCESS-CM2 (187)
CNRM-ESM2-1 (178)
NorCPM1 (172)
EC-Earth3-CC (134)
CESM2 (132)
FGOALS-g3 (120)
NorESM2-LM (114)
E3SM-1-0 (111)
MPI-ESM1-2-HR (110)
E3SM-2-0 (104)
HadGEM3-GC31-LL (104)
GISS-E2-2-G (99)
CESM2-WACCM (96)
INM-CM5-0 (96)
NESM3 (88)
KACE-1-0-G (86)
CMCC-CM2-SR5 (84)
EC-Earth3-Veg-LR (66)
HadGEM3-GC31-MM (62)
GFDL-ESM4 (58)
AWI-CM-1-1-MR (56)
BCC-CSM2-MR (46)
NorESM2-MM (44)
FGOALS-f3-L (40)
ICON-ESM-LR (40)
CAS-ESM2-0 (39)
MIROC-ES2H (39)
CAMS-CSM1-0 (37)
CanESM5-CanOE (36)
FIO-ESM-2-0 (34)
BCC-ESM1 (30)
CESM2-FV2 (30)
E3SM-2-0-NARRM (30)
E3SM-1-1 (29)
GFDL-CM4 (29)
KIOST-ESM (29)
TaiESM1 (26)
GISS-E2-2-H (25)
CESM2-WACCM-FV2 (24)
CMCC-ESM2 (24)
EC-Earth3-AerChem (24)
INM-CM4-8 (24)
MPI-ESM-1-2-HAM (24)
CIESM (23)
CNRM-CM6-1-HR (22)
IITM-ESM (21)
MCM-UA-1-0 (16)
E3SM-1-1-ECA (11)
E3SM-2-1 (10)
SAM0-UNICON (10)
AWI-ESM-1-1-LR (8)
CMCC-CM2-HR4 (8)
GISS-E2-1-G-CC (8)
IPSL-CM5A2-INCA (6)
IPSL-CM6A-LR-INCA (4)
UKESM1-1-LL (4)
GISS-E3-G (3)
AWI-ESM-1-REcoM (2)

In [9]:
# Conexão ao ESGF LLNL
conn = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False)

# Modelos e variáveis de interesse
modelos = ["EC-Earth3", "CanESM5", "MPI-ESM1-2-LR"]
variaveis = ["tas", "tasmin", "tasmax", "tos", "ps", "pr"]
experimentos = ["historical", "ssp245", "ssp585"]

# Dicionário para armazenar os resultados
resultados = defaultdict(lambda: defaultdict(set))

# Loop por variável e modelo
for var in variaveis:
    for modelo in modelos:
        ctx = conn.new_context(
            project="CMIP6",
            experiment_id=experimentos,
            variable_id=var,
            source_id=modelo,
            frequency="mon",
            replica=False,
            latest=True
        )
        results = list(ctx.search(ignore_facet_check=True))
        for d in results:
            experimento = d.json.get("experiment_id", [None])[0]
            if experimento:
                resultados[modelo][experimento].add(var)

# Exibição dos resultados
for modelo, exps in resultados.items():
    print(f"\nModelo: {modelo}")
    for exp, vars in exps.items():
        print(f"  Experimento: {exp} → Variáveis: {sorted(vars)}")


Modelo: EC-Earth3
  Experimento: ssp585 → Variáveis: ['pr', 'ps', 'tas', 'tasmax', 'tasmin', 'tos']

Modelo: CanESM5
  Experimento: ssp585 → Variáveis: ['pr', 'ps', 'tas', 'tasmax', 'tasmin', 'tos']

Modelo: MPI-ESM1-2-LR
  Experimento: ssp585 → Variáveis: ['pr', 'ps', 'tas', 'tasmax', 'tasmin', 'tos']
